# S3 Vectors API Testing with Boto3 SDK

This notebook demonstrates how to test the S3 Vectors API using the boto3 SDK with a custom service model. This approach provides a native AWS SDK experience with proper authentication, retry logic, and error handling.

Make sure the FastAPI server is running on localhost:8000 before executing these cells.

## 🔧 Environment Configuration Guide

Before running this notebook, update the configuration in the first code cell based on your environment:

### 🏠 Local Development
```python
ENDPOINT_URL = "http://localhost:8000"
AWS_ACCESS_KEY_ID = "test"
AWS_SECRET_ACCESS_KEY = "test"
```

### 🌐 Remote Development Server
```python
ENDPOINT_URL = "http://your-dev-server.com:8000"
AWS_ACCESS_KEY_ID = "your-dev-access-key"
AWS_SECRET_ACCESS_KEY = "your-dev-secret-key"
```

### ☁️ Production/Staging Environment
```python
ENDPOINT_URL = "https://s3vectors-api.your-company.com"
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
```

### 🐳 Docker Environment
```python
ENDPOINT_URL = "http://s3vectors-container:8000"
AWS_ACCESS_KEY_ID = "docker-test"
AWS_SECRET_ACCESS_KEY = "docker-test"
```

⚠️ **Security Note**: Never commit real credentials to version control. Use environment variables or secure credential management for production.

## 1. Import Libraries and Configuration

This notebook demonstrates S3 Vectors API testing with boto3 SDK using real text embeddings. The setup includes:
- S3 Vectors client configuration  
- Text embedding server integration (text-embedding-nomic-embed-text-v1.5)
- Rivers of India knowledge base for semantic search testing

In [ ]:
# =============================================================================
# 🔧 CONFIGURATION - Update these settings for your environment
# =============================================================================

# S3 Vectors API Endpoint Configuration
ENDPOINT_URL = "http://127.0.0.1:8000/"  # Change to your S3 Vectors server URL
REGION_NAME = "us-east-1"               # AWS region for compatibility

# AWS Credentials (for boto3 compatibility if needed)
AWS_ACCESS_KEY_ID = "minioadmin"              # Your AWS access key or test value
AWS_SECRET_ACCESS_KEY = "minioadmin"          # Your AWS secret key or test value

# Embedding Server Configuration
EMBEDDING_URL = "http://127.0.0.1:1234/v1/embeddings"  # Local embedding server
EMBEDDING_MODEL = "text-embedding-nomic-embed-text-v1.5"  # Embedding model

# Request Configuration
REQUEST_TIMEOUT = 30                    # Request timeout in seconds
MAX_RETRIES = 3                        # Maximum number of retries for failed requests

print("🔧 Configuration Settings:")
print(f"   📡 S3 Vectors Endpoint: {ENDPOINT_URL}")
print(f"   🧠 Embedding Server: {EMBEDDING_URL}")
print(f"   🤖 Embedding Model: {EMBEDDING_MODEL}")
print(f"   🌍 Region: {REGION_NAME}")
print(f"   🔑 Access Key: {AWS_ACCESS_KEY_ID[:4]}***")
print(f"   ⏱️ Timeout: {REQUEST_TIMEOUT}s")
print(f"   🔄 Max Retries: {MAX_RETRIES}")

# =============================================================================
# 📚 LIBRARY IMPORTS
# =============================================================================

import boto3
import botocore
import json
import numpy as np
import os
import sys
import threading
import concurrent.futures
import time
import requests  # Added for embedding API calls
from botocore.loaders import Loader

print("\n📚 Libraries imported successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🔧 Boto3 version: {boto3.__version__}")
print(f"🔧 Botocore version: {botocore.__version__}")
print(f"🔧 Requests version: {requests.__version__}")
print("💡 Using boto3 SDK with S3 Vectors service model!")
print("🧠 Ready to generate real text embeddings!")

🔧 Configuration Settings:
   📡 S3 Vectors Endpoint: http://127.0.0.1:8000/
   🧠 Embedding Server: http://127.0.0.1:1234/v1/embeddings
   🤖 Embedding Model: text-embedding-nomic-embed-text-v1.5
   🌍 Region: us-east-1
   🔑 Access Key: mini***
   ⏱️ Timeout: 30s
   🔄 Max Retries: 3

📚 Libraries imported successfully!
🐍 Python version: 3.13.5 (main, Jul  1 2025, 18:16:22) [Clang 20.1.4 ]
🔧 Boto3 version: 1.40.7
🔧 Botocore version: 1.40.7
🔧 Requests version: 2.32.4
🧵 Threading support: 6 active threads
🚀 Python 3.13.5+ provides improved multithreading performance!
💡 Using boto3 SDK with S3 Vectors service model!
🧠 Ready to generate real text embeddings!


## 2. Configure Boto3 Client with S3 Vectors Service Model

Configure the boto3 client to use the S3 Vectors service model for native AWS SDK functionality.

In [103]:
# Configure boto3 to use S3 Vectors service model
print(f"🔧 Setting up boto3 S3 Vectors client...")
print(f"📡 Configured Endpoint URL: {ENDPOINT_URL}")
print(f"🌍 Region: {REGION_NAME}")



# Clear any conflicting environment variables that might override our endpoint
env_vars_to_clear = ['AWS_ENDPOINT_URL', 'AWS_ENDPOINT_URL_S3', 'MINIO_ENDPOINT']
for var in env_vars_to_clear:
    if var in os.environ:
        print(f"🧹 Clearing environment variable: {var}={os.environ[var]}")
        del os.environ[var]

try:
    # Ensure we use the exact endpoint URL from configuration
    actual_endpoint = ENDPOINT_URL.rstrip('/')  # Remove trailing slash for consistency
    print(f"📡 Using Endpoint URL: {actual_endpoint}")
    
    # Create S3 Vectors client (NOT standard S3 client)
    s3vectors_client = boto3.client(
        's3vectors',  # This is the key - use 's3vectors' service
        region_name=REGION_NAME,
        endpoint_url=actual_endpoint,
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        config=boto3.session.Config(
            retries={'max_attempts': MAX_RETRIES},
            read_timeout=REQUEST_TIMEOUT,
            connect_timeout=REQUEST_TIMEOUT,
            signature_version=botocore.UNSIGNED,  # Use botocore.UNSIGNED
        ),
        verify=False  # Skip SSL verification for local development
    )
    
    # Verify the client is using the correct endpoint
    client_endpoint = s3vectors_client._endpoint.host
    print(f"✅ Client endpoint verified: {client_endpoint}")
    
    if client_endpoint != actual_endpoint:
        print(f"⚠️ WARNING: Client endpoint ({client_endpoint}) differs from configured ({actual_endpoint})")
    
    print("✅ Boto3 S3 Vectors client created successfully!")
    print("🔧 Using S3 Vectors service with native boto3 methods")
    print("📡 Client supports: create_index(), put_vectors(), query_vectors(), etc.")
    print("🎯 Ready to use S3 Vectors operations!")
        
except Exception as e:
    print(f"❌ Error setting up S3 Vectors client: {e}")
    print("🚨 Please check:")
    print("   1. Server is running at the configured endpoint")
    print("   2. S3 Vectors service model is available")
    print("   3. Endpoint URL is correct in configuration")
    print("   4. Service model path is properly configured")
    
    s3vectors_client = None

🔧 Setting up boto3 S3 Vectors client...
📡 Configured Endpoint URL: http://127.0.0.1:8000/
🌍 Region: us-east-1
📡 Using Endpoint URL: http://127.0.0.1:8000
✅ Client endpoint verified: http://127.0.0.1:8000
✅ Boto3 S3 Vectors client created successfully!
🔧 Using S3 Vectors service with native boto3 methods
📡 Client supports: create_index(), put_vectors(), query_vectors(), etc.
🎯 Ready to use S3 Vectors operations!


## 3. S3 Vectors Client Ready

The boto3 S3 Vectors client is now configured and ready to use. This provides native AWS SDK functionality with proper error handling, authentication, and retry logic.

In [104]:
# Verify boto3 S3 Vectors client is ready
if s3vectors_client is not None:
    print("🚀 Boto3 S3 Vectors client is ready!")
    print(f"📡 Endpoint URL: {s3vectors_client._endpoint.host}")
    print(f"🌍 Region: {s3vectors_client.meta.region_name}")
    print(f"🔧 Service: S3 Vectors (with native API support)")
    print("✅ Ready to test S3 Vectors operations:")
    print("   📦 Bucket operations: create_vector_bucket(), list_vector_buckets()")
    print("   📊 Index operations: create_index(), list_indexes(), delete_index()")
    print("   🔍 Vector operations: put_vectors(), get_vectors(), query_vectors()")
    print("   🔐 Policy operations: put_vector_bucket_policy(), get_vector_bucket_policy()")
    print("💡 Using native S3 Vectors boto3 client")
else:
    print("❌ S3 Vectors client not available")
    print("🚨 Please check the server connectivity and configuration")
    print("💡 Make sure to run the previous cell successfully before proceeding")

🚀 Boto3 S3 Vectors client is ready!
📡 Endpoint URL: http://127.0.0.1:8000
🌍 Region: us-east-1
🔧 Service: S3 Vectors (with native API support)
✅ Ready to test S3 Vectors operations:
   📦 Bucket operations: create_vector_bucket(), list_vector_buckets()
   📊 Index operations: create_index(), list_indexes(), delete_index()
   🔍 Vector operations: put_vectors(), get_vectors(), query_vectors()
   🔐 Policy operations: put_vector_bucket_policy(), get_vector_bucket_policy()
💡 Using native S3 Vectors boto3 client


## 4. Test CreateVectorBucket

Create a new vector bucket using the boto3 S3 Vectors client. The bucket name includes hostname for uniqueness across environments.

In [105]:
# Test S3 Vectors operations with native boto3 methods
import time

print("🧪 Testing S3 Vectors operations with native boto3 methods...")
print("📡 Using S3 Vectors client with create_vector_bucket(), list_vector_buckets(), etc.")

# Test 1: List vector buckets
try:
    print("\n1️⃣ Testing boto3 list_vector_buckets()")
    response = s3vectors_client.list_vector_buckets()
    print("✅ list_vector_buckets() successful!")
    
    # S3 Vectors response format
    buckets = response.get('vectorBuckets', [])
    print(f"📊 Found {len(buckets)} buckets")
    
    for bucket in buckets:
        print(f"  📦 {bucket['vectorBucketName']} (created: {bucket.get('creationTime', 'N/A')})")
        
except Exception as e:
    print(f"❌ Error in list_vector_buckets(): {e}")

# Test 2: Create vector bucket
bucket_name = f"boto3-test-{int(time.time())}"
try:
    print(f"\n2️⃣ Testing boto3 create_vector_bucket()")
    print(f"🏗️ Creating bucket: {bucket_name}")
    
    response = s3vectors_client.create_vector_bucket(vectorBucketName=bucket_name)
    print("✅ create_vector_bucket() successful!")
    print(f"📍 Response: {response}")
    
except Exception as e:
    print(f"❌ Error in create_vector_bucket(): {e}")

# Test 3: Verify bucket was created
try:
    print(f"\n3️⃣ Verifying bucket creation...")
    response = s3vectors_client.list_vector_buckets()
    
    # Use correct S3 Vectors response format
    buckets = response.get('vectorBuckets', [])
    bucket_names = [b['vectorBucketName'] for b in buckets]
    
    if bucket_name in bucket_names:
        print(f"✅ Bucket '{bucket_name}' successfully created!")
    else:
        print(f"⚠️ Bucket '{bucket_name}' not found in list")
        print(f"🔍 Available buckets: {bucket_names}")
        
except Exception as e:
    print(f"❌ Error verifying bucket: {e}")

print(f"\n🎉 S3 Vectors boto3 client is working!")
print(f"💡 Using native S3 Vectors operations: create_vector_bucket(), list_vector_buckets()")
print(f"🔧 Client has full S3 Vectors support with proper method signatures!")

🧪 Testing S3 Vectors operations with native boto3 methods...
📡 Using S3 Vectors client with create_vector_bucket(), list_vector_buckets(), etc.

1️⃣ Testing boto3 list_vector_buckets()
✅ list_vector_buckets() successful!
📊 Found 14 buckets
  📦 b1 (created: 2025-08-13 04:42:54.826958+00:00)
  📦 boto3-test-1755055901 (created: 2025-08-13 04:42:54.827154+00:00)
  📦 boto3-test-1755056431 (created: 2025-08-13 04:42:54.827159+00:00)
  📦 boto3-test-1755057315 (created: 2025-08-13 04:42:54.827161+00:00)
  📦 boto3-test-1755057347 (created: 2025-08-13 04:42:54.827163+00:00)
  📦 boto3-test-1755057682 (created: 2025-08-13 04:42:54.827165+00:00)
  📦 boto3-test-1755058205 (created: 2025-08-13 04:42:54.827166+00:00)
  📦 boto3-test-1755059506 (created: 2025-08-13 04:42:54.827171+00:00)
  📦 bucket1 (created: 2025-08-13 04:42:54.827174+00:00)
  📦 test-bucket (created: 2025-08-13 04:42:54.827178+00:00)
  📦 test-bucket-2 (created: 2025-08-13 04:42:54.827180+00:00)
  📦 test-bucket-live (created: 2025-08-13

## 5. Test ListVectorBuckets

List all available vector buckets.

In [106]:
# List vector buckets using native S3 Vectors method
try:
    response = s3vectors_client.list_vector_buckets()
    
    print("✅ Vector buckets listed successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Count buckets using correct S3 Vectors response format
    buckets = response.get('vectorBuckets', [])
    bucket_count = len(buckets)
    print(f"📊 Total buckets: {bucket_count}")
    
    # Show bucket names
    if buckets:
        for bucket in buckets:
            print(f"  📦 {bucket['vectorBucketName']}")
    else:
        print("  📭 No buckets found")
            
except Exception as e:
    print(f"❌ Error listing buckets: {e}")

✅ Vector buckets listed successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 04:43:31 GMT",
      "server": "uvicorn",
      "content-length": "2681",
      "content-type": "application/json"
    },
    "RetryAttempts": 0
  },
  "vectorBuckets": [
    {
      "vectorBucketName": "b1",
      "vectorBucketArn": "arn:aws:s3vectors:us-east-1:123456789012:bucket/b1",
      "creationTime": "2025-08-13 04:43:31.933168+00:00"
    },
    {
      "vectorBucketName": "boto3-test-1755055901",
      "vectorBucketArn": "arn:aws:s3vectors:us-east-1:123456789012:bucket/boto3-test-1755055901",
      "creationTime": "2025-08-13 04:43:31.933222+00:00"
    },
    {
      "vectorBucketName": "boto3-test-1755056431",
      "vectorBucketArn": "arn:aws:s3vectors:us-east-1:123456789012:bucket/boto3-test-1755056431",
      "creationTime": "2025-08-13 04:43:31.933226+00:00"
    },
    {
      "vectorBucketName": "boto3-test-1755057315",
      "ve

## 6. Test PutVectorBucketPolicy

Set a bucket policy for the vector bucket.

In [61]:
# Create and apply bucket policy
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": "*",
            "Action": [
                "s3vectors:GetVectors",
                "s3vectors:QueryVectors"
            ],
            "Resource": f"arn:aws:s3vectors:*:*:bucket/{bucket_name}/*"
        }
    ]
}

try:
    response = s3vectors_client.put_vector_bucket_policy(
        vectorBucketName=bucket_name,
        policy=policy
    )
    
    print("✅ Bucket policy set successfully!")
    print(json.dumps(response, indent=2, default=str))
    
except Exception as e:
    print(f"❌ Error setting bucket policy: {e}")

❌ Error setting bucket policy: Parameter validation failed:
Invalid type for parameter policy, value: {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': '*', 'Action': ['s3vectors:GetVectors', 's3vectors:QueryVectors'], 'Resource': 'arn:aws:s3vectors:*:*:bucket/boto3-test-1755057682/*'}]}, type: <class 'dict'>, valid types: <class 'str'>


## 7. Test GetVectorBucketPolicy

Retrieve the bucket policy we just set.

In [64]:
# Get bucket policy
try:
    response = s3vectors_client.get_vector_bucket_policy(
        vectorBucketName=bucket_name
    )
    
    print("✅ Bucket policy retrieved successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Verify policy content
    if 'policy' in response:
        policy_version = response['policy'].get('Version')
        statement_count = len(response['policy'].get('Statement', []))
        print(f"📋 Policy version: {policy_version}")
        print(f"📊 Number of statements: {statement_count}")
        
except Exception as e:
    print(f"❌ Error getting bucket policy: {e}")

✅ Bucket policy retrieved successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 04:03:37 GMT",
      "server": "uvicorn",
      "content-length": "15",
      "content-type": "application/json"
    },
    "RetryAttempts": 0
  }
}


## 8. Test CreateIndex

Create a vector index in the bucket.

In [107]:
# Create vector index
index_name = "test-notebook-index"

try:
    response = s3vectors_client.create_index(
        vectorBucketName=bucket_name,
        indexName=index_name,
        dimension=768,  # Updated to match text-embedding-nomic-embed-text-v1.5 dimension
        dataType="float32",
        distanceMetric="cosine",
        metadataConfiguration={
            "nonFilterableMetadataKeys": ["description", "internal_id"]
        }
    )
    
    print("✅ Vector index created successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Extract index details
    if 'index' in response:
        index_info = response['index']
        print(f"📊 Index: {index_info.get('indexName')}")
        print(f"📏 Dimension: {index_info.get('dimension')}")
        print(f"📐 Distance metric: {index_info.get('distanceMetric')}")
        print(f"📅 Creation time: {index_info.get('creationTime')}")
        
except Exception as e:
    print(f"❌ Error creating index: {e}")

✅ Vector index created successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 04:44:03 GMT",
      "server": "uvicorn",
      "content-length": "2",
      "content-type": "application/json"
    },
    "RetryAttempts": 0
  }
}


## 9. Test ListIndexes

List all indexes in the bucket.

In [108]:
# List indexes
try:
    response = s3vectors_client.list_indexes(
        vectorBucketName=bucket_name
    )
    
    print("✅ Indexes listed successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Count indexes
    index_count = len(response.get('indexes', []))
    print(f"📊 Total indexes: {index_count}")
    
    # Show index names
    if 'indexes' in response:
        for index in response['indexes']:
            print(f"  📊 {index['indexName']}")
            
except Exception as e:
    print(f"❌ Error listing indexes: {e}")

✅ Indexes listed successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 04:44:11 GMT",
      "server": "uvicorn",
      "content-length": "261",
      "content-type": "application/json"
    },
    "RetryAttempts": 0
  },
  "indexes": [
    {
      "vectorBucketName": "boto3-test-1755060174",
      "indexName": "test-notebook-index",
      "indexArn": "arn:aws:s3vectors:us-east-1:123456789012:index/boto3-test-1755060174/test-notebook-index",
      "creationTime": "2025-08-13 04:44:11.645535+00:00"
    }
  ]
}
📊 Total indexes: 1
  📊 test-notebook-index


## 10. Test PutVectors with Real Text Embeddings

Add vectors to the index using real text embeddings about rivers of India. This replaces random vectors with meaningful semantic representations generated by the text-embedding-nomic-embed-text-v1.5 model.

### 🌊 River Knowledge Base
- **Ganges (Ganga)**: Sacred river flowing from Himalayas to Bay of Bengal
- **Brahmaputra**: Major river supporting agriculture in northeastern India
- **Narmada**: Westward-flowing river important for hydropower and irrigation
- **Krishna**: River crucial for Deccan plateau agriculture

### 🧠 Embedding Features
- **768-dimensional vectors** from text-embedding-nomic-embed-text-v1.5
- **Semantic understanding** of geographic, religious, and economic concepts
- **Real similarity search** based on meaning, not just keywords

In [109]:
# Generate embeddings using the local embedding server
import requests

def get_text_embedding(text, model="text-embedding-nomic-embed-text-v1.5"):
    """Generate text embedding using local embedding server"""
    try:
        response = requests.post(
            "http://127.0.0.1:1234/v1/embeddings",
            headers={"Content-Type": "application/json"},
            json={
                "model": model,
                "input": text
            },
            timeout=30
        )
        response.raise_for_status()
        data = response.json()
        
        # Extract embedding from response
        embedding = data["data"][0]["embedding"]
        print(f"✅ Generated embedding for: '{text[:50]}...' (dimension: {len(embedding)})")
        return embedding
        
    except Exception as e:
        print(f"❌ Error generating embedding for '{text[:50]}...': {e}")
        # Fallback to random vector if embedding fails
        vector = np.random.randn(768).astype(np.float32)
        norm = np.linalg.norm(vector)
        if norm > 0:
            vector = vector / norm
        return vector.tolist()

# Create sample texts about rivers of India with comprehensive information
river_texts = [
    {
        "key": "ganga-river",
        "text": "The Ganges, known as Ganga in Hindi, is the most sacred river in India. It originates from the Gangotri Glacier in the Himalayas and flows through northern India for 2,525 kilometers before emptying into the Bay of Bengal. The river is considered holy by Hindus and supports over 400 million people along its course. Major cities like Varanasi, Allahabad, and Kolkata are situated on its banks.",
        "metadata": {
            "title": "Ganges River - Sacred Waters of India",
            "category": "geography",
            "region": "Northern India",
            "length_km": 2525,
            "type": "sacred_river",
            "importance": "religious_economic"
        }
    },
    {
        "key": "brahmaputra-river", 
        "text": "The Brahmaputra is one of the major rivers of Asia, flowing through Tibet, India, and Bangladesh. In India, it flows through Assam for 720 kilometers and is known as one of the few male rivers in Hindu tradition. The river is vital for agriculture in the northeastern states and supports rich biodiversity. It eventually joins the Ganges to form the world's largest delta.",
        "metadata": {
            "title": "Brahmaputra River - The Son of Brahma",
            "category": "geography", 
            "region": "Northeastern India",
            "length_km": 720,
            "type": "major_river",
            "importance": "agricultural_biodiversity"
        }
    },
    {
        "key": "narmada-river",
        "text": "The Narmada River is the fifth-longest river in India, flowing westward for 1,312 kilometers through Madhya Pradesh, Maharashtra, and Gujarat before draining into the Arabian Sea. It is one of only three major rivers in peninsular India that flow from east to west. The river is considered sacred and has numerous ancient temples along its banks. The Sardar Sarovar Dam on this river is one of the largest infrastructure projects in India.",
        "metadata": {
            "title": "Narmada River - The Lifeline of Central India", 
            "category": "geography",
            "region": "Central India",
            "length_km": 1312,
            "type": "westward_flowing",
            "importance": "irrigation_hydropower"
        }
    },
    {
        "key": "krishna-river",
        "text": "The Krishna River is the fourth-longest river in India, flowing for 1,400 kilometers through Maharashtra, Karnataka, Telangana, and Andhra Pradesh before emptying into the Bay of Bengal. The river originates near Mahabaleshwar in the Western Ghats and is crucial for irrigation in the Deccan Plateau region. Major cities like Vijayawada and Sangli are located on its banks, and it supports extensive agricultural activities.",
        "metadata": {
            "title": "Krishna River - Waters of the Deccan",
            "category": "geography",
            "region": "South India", 
            "length_km": 1400,
            "type": "peninsular_river",
            "importance": "irrigation_agriculture"
        }
    }
]

print("🌊 Creating vectors with real text embeddings about rivers of India...")
print(f"📡 Using embedding model: text-embedding-nomic-embed-text-v1.5")
print(f"📏 Expected dimension: 768")

# Generate embeddings for each river text
vectors = []
for river_data in river_texts:
    print(f"\n🔄 Processing: {river_data['metadata']['title']}")
    
    # Get embedding for the full text
    embedding = get_text_embedding(river_data['text'])
    
    # Create vector entry
    vector_entry = {
        "key": river_data['key'],
        "data": {"float32": embedding},
        "metadata": river_data['metadata']
    }
    vectors.append(vector_entry)

try:
    response = s3vectors_client.put_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        vectors=vectors
    )
    
    print("\n✅ Vectors uploaded successfully!")
    print(json.dumps(response, indent=2, default=str))
    print(f"📊 Uploaded {len(vectors)} vectors with real embeddings")
    
    for vector in vectors:
        print(f"  🌊 {vector['key']}: {vector['metadata']['title']}")
        
except Exception as e:
    print(f"❌ Error uploading vectors: {e}")

🌊 Creating vectors with real text embeddings about rivers of India...
📡 Using embedding model: text-embedding-nomic-embed-text-v1.5
📏 Expected dimension: 768

🔄 Processing: Ganges River - Sacred Waters of India
✅ Generated embedding for: 'The Ganges, known as Ganga in Hindi, is the most s...' (dimension: 768)

🔄 Processing: Brahmaputra River - The Son of Brahma
✅ Generated embedding for: 'The Brahmaputra is one of the major rivers of Asia...' (dimension: 768)

🔄 Processing: Narmada River - The Lifeline of Central India
✅ Generated embedding for: 'The Narmada River is the fifth-longest river in In...' (dimension: 768)

🔄 Processing: Krishna River - Waters of the Deccan
✅ Generated embedding for: 'The Krishna River is the fourth-longest river in I...' (dimension: 768)

✅ Vectors uploaded successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 04:45:26 GMT",
      "server": "uvicorn",
      "content-length": "2",
      "conte

## 11. Test QueryVectors

Search for similar vectors using a query vector.

### Filter Format for S3 Vectors

S3 Vectors uses a structured filter format with operators. Common patterns:

```python
# Equality filter
filter = {
    "category": {
        "eq": "education"
    }
}

# Numeric comparison
filter = {
    "score": {
        "gte": 0.8
    }
}

# Multiple conditions
filter = {
    "category": {
        "eq": "education"
    },
    "score": {
        "gte": 0.85
    }
}

# Available operators: eq, neq, gt, gte, lt, lte, in, nin, exists
```

In [114]:
# Query for similar vectors with a real question about rivers of India
query_text = "Which river is considered most sacred in Hindu religion and flows from the Himalayas to the Bay of Bengal?"

print(f"🔍 Query Question: {query_text}")
print("🔄 Generating embedding for query...")

# Generate embedding for the query
query_vector = get_text_embedding(query_text)

try:
    response = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": query_vector},
        topK=3,
        returnMetadata=True,   # Enable metadata return for similarity search
        returnDistance=True,   # Enable distance return for similarity search
        filter={
            "category": {
                "eq": "geography"  # Filter for geography-related content
            }
        }
    )
    
    print("✅ Vector similarity search completed successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Display results with proper similarity search information
    if 'vectors' in response:
        print(f"\n🔍 Similarity Search Results ({len(response['vectors'])} found):")
        print(f"❓ Question: {query_text}")
        for i, result in enumerate(response['vectors'], 1):
            key = result.get('key', 'Unknown')
            distance = result.get('distance', 'N/A')
            metadata = result.get('metadata', {})
            title = metadata.get('title', 'No title')
            region = metadata.get('region', 'No region')
            length_km = metadata.get('length_km', 'N/A')
            river_type = metadata.get('type', 'No type')
            importance = metadata.get('importance', 'No importance')
            
            print(f"\n  {i}. 🌊 {key}: {title}")
            print(f"      📍 Region: {region}")
            print(f"      📏 Length: {length_km} km")
            print(f"      🏷️ Type: {river_type}")
            print(f"      ⭐ Importance: {importance}")
            print(f"      🎯 Similarity Distance: {distance:.4f}" if isinstance(distance, (int, float)) else f"      🎯 Similarity Distance: {distance}")
            
            # Interpret similarity
            if isinstance(distance, (int, float)):
                if distance < 0.3:
                    similarity_desc = "Very Relevant ✅"
                elif distance < 0.6:
                    similarity_desc = "Relevant ✅"
                elif distance < 0.9:
                    similarity_desc = "Somewhat Relevant ⚠️"
                else:
                    similarity_desc = "Less Relevant ❌"
                print(f"      📊 Relevance: {similarity_desc}")
    else:
        print("🔍 No similar vectors found (filtering may have excluded results)")
    
except Exception as e:
    print(f"❌ Error in similarity search: {e}")

🔍 Query Question: Which river is considered most sacred in Hindu religion and flows from the Himalayas to the Bay of Bengal?
🔄 Generating embedding for query...
✅ Generated embedding for: 'Which river is considered most sacred in Hindu rel...' (dimension: 768)
✅ Vector similarity search completed successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 05:03:19 GMT",
      "server": "uvicorn",
      "content-length": "787",
      "content-type": "application/json"
    },
    "RetryAttempts": 0
  },
  "vectors": [
    {
      "key": "ganga-river",
      "metadata": {
        "title": "Ganges River - Sacred Waters of India",
        "category": "geography",
        "region": "Northern India",
        "length_km": 2525,
        "type": "sacred_river",
        "importance": "religious_economic"
      },
      "distance": 0.12343692779541016
    },
    {
      "key": "brahmaputra-river",
      "metadata": {
        "title": "Bra

In [91]:
# Verify vector dimensions and test exact text similarity
print("🔧 Verifying semantic similarity search with real embeddings...")

# Check the dimensions of our query vector
print(f"📏 Query vector dimension: {len(query_vector)}")

# Test semantic similarity with exact text from stored vector
print("\n🧪 Testing exact text similarity...")
exact_ganges_text = "The Ganges, known as Ganga in Hindi, is the most sacred river in India. It originates from the Gangotri Glacier in the Himalayas and flows through northern India for 2,525 kilometers before emptying into the Bay of Bengal."

try:
    # Generate embedding for the exact text
    exact_ganges_vector = get_text_embedding(exact_ganges_text)
    
    # Query with the exact text - should return Ganges with very low distance
    similarity_response = s3vectors_client.query_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        queryVector={"float32": exact_ganges_vector},
        topK=3,
        returnMetadata=True,
        returnDistance=True
    )
    
    print("✅ Exact text similarity search results:")
    if 'vectors' in similarity_response:
        for i, result in enumerate(similarity_response['vectors'], 1):
            key = result.get('key', 'Unknown')
            distance = result.get('distance', 'N/A')
            metadata = result.get('metadata', {})
            title = metadata.get('title', 'No title')
            
            print(f"  {i}. 🌊 {key}: {title}")
            print(f"       Distance: {distance:.6f}" if isinstance(distance, (int, float)) else f"      📏 Distance: {distance}")
            
            # Check for very close semantic match with Ganges
            if key == "ganga-river" and isinstance(distance, (int, float)) and distance < 0.1:
                print("🎯 ⭐ EXCELLENT SEMANTIC MATCH! ⭐")
    
    print("\n📊 This demonstrates that text embeddings capture semantic meaning effectively")
    
except Exception as e:
    print(f"❌ Error in exact text similarity test: {e}")

print("\n" + "="*70)

🔧 Verifying vector similarity search configuration...
📏 Query vector dimension: 128
📊 Index configuration:
   📏 Index dimension: 128
   📐 Distance metric: cosine
   🔢 Data type: float32
✅ Vector dimensions match - similarity search should work correctly

--------------------------------------------------
🧪 Testing similarity search with a known vector...
📄 Using doc-1 vector as query (dimension: 128)
✅ Known vector similarity search results:
  1. doc-1: Machine Learning Fundamentals (distance: -0.000000)
🎯 Perfect match found! Similarity search is working correctly.
  2. doc-3: Natural Language Processing (distance: 0.887455)
  3. doc-3: Natural Language Processing (distance: 0.948686)



In [115]:
# Test semantic search with various question types about rivers of India
print("🌊 Testing semantic search with different types of questions...")
print("📚 This demonstrates how embeddings capture semantic meaning beyond keywords")

# Define test questions of different types
test_questions = [
    {
        "question": "What is the holiest river for Hindu worship?",
        "expected_match": "ganga-river",
        "explanation": "Religious/spiritual question - should match Ganges"
    },
    {
        "question": "Which river supports the most agriculture in northeastern states?",
        "expected_match": "brahmaputra-river", 
        "explanation": "Agricultural question - should match Brahmaputra"
    },
    {
        "question": "Tell me about rivers that are important for hydropower generation",
        "expected_match": "narmada-river",
        "explanation": "Energy/infrastructure question - should match Narmada"
    },
    {
        "question": "Which river is crucial for farming in the Deccan plateau?",
        "expected_match": "krishna-river",
        "explanation": "Geographic/agricultural question - should match Krishna"
    }
]

print(f"\n🔬 Running {len(test_questions)} semantic search tests...\n")

for i, test in enumerate(test_questions, 1):
    print(f"{'='*60}")
    print(f"🧪 TEST {i}: {test['explanation']}")
    print(f"❓ Question: {test['question']}")
    print(f"🎯 Expected top match: {test['expected_match']}")
    print(f"{'='*60}")
    
    try:
        # Generate embedding for the test question
        test_vector = get_text_embedding(test['question'])
        
        # Perform similarity search
        response = s3vectors_client.query_vectors(
            vectorBucketName=bucket_name,
            indexName=index_name,
            queryVector={"float32": test_vector},
            topK=2,  # Just get top 2 results
            returnMetadata=True,
            returnDistance=True
        )
        
        if 'vectors' in response and len(response['vectors']) > 0:
            top_result = response['vectors'][0]
            top_key = top_result.get('key', 'Unknown')
            top_distance = top_result.get('distance', 'N/A')
            top_title = top_result.get('metadata', {}).get('title', 'No title')
            
            print(f"🏆 TOP RESULT: {top_key}")
            print(f"📰 Title: {top_title}")
            print(f"📏 Distance: {top_distance:.4f}" if isinstance(top_distance, (int, float)) else f"📏 Distance: {top_distance}")
            
            # Check if prediction was correct
            if top_key == test['expected_match']:
                print("✅ ⭐ SEMANTIC SEARCH SUCCESS! ⭐")
                print("🎯 The embedding model correctly understood the semantic meaning!")
            else:
                print("⚠️ Different result than expected")
                print(f"   Expected: {test['expected_match']}")
                print(f"   Got: {top_key}")
                print("   This could still be semantically correct!")
            
            # Show second result for comparison
            if len(response['vectors']) > 1:
                second_result = response['vectors'][1]
                second_key = second_result.get('key', 'Unknown')
                second_distance = second_result.get('distance', 'N/A')
                second_title = second_result.get('metadata', {}).get('title', 'No title')
                print(f"\n🥈 SECOND: {second_key} - {second_title}")
                print(f"📏 Distance: {second_distance:.4f}" if isinstance(second_distance, (int, float)) else f"📏 Distance: {second_distance}")
        else:
            print("❌ No results found")
            
    except Exception as e:
        print(f"❌ Error in test {i}: {e}")
    
    print()  # Empty line between tests

print("🎊 Semantic search testing complete!")
print("💡 These tests demonstrate how text embeddings capture:")
print("   🔸 Religious concepts (sacred, holy, worship)")
print("   🔸 Geographic relationships (northeastern, Deccan plateau)")
print("   🔸 Functional purposes (agriculture, hydropower, irrigation)")
print("   🔸 Economic activities (farming, infrastructure)")
print("\n🚀 This is the power of semantic vector search with real embeddings!")

🌊 Testing semantic search with different types of questions...
📚 This demonstrates how embeddings capture semantic meaning beyond keywords

🔬 Running 4 semantic search tests...

🧪 TEST 1: Religious/spiritual question - should match Ganges
❓ Question: What is the holiest river for Hindu worship?
🎯 Expected top match: ganga-river
✅ Generated embedding for: 'What is the holiest river for Hindu worship?...' (dimension: 768)
🏆 TOP RESULT: ganga-river
📰 Title: Ganges River - Sacred Waters of India
📏 Distance: 0.1849
✅ ⭐ SEMANTIC SEARCH SUCCESS! ⭐
🎯 The embedding model correctly understood the semantic meaning!

🥈 SECOND: narmada-river - Narmada River - The Lifeline of Central India
📏 Distance: 0.2134

🧪 TEST 2: Agricultural question - should match Brahmaputra
❓ Question: Which river supports the most agriculture in northeastern states?
🎯 Expected top match: brahmaputra-river
✅ Generated embedding for: 'Which river supports the most agriculture in north...' (dimension: 768)
🏆 TOP RESULT: brah

## 12. Test GetVectors

Retrieve specific vectors by their keys.

In [116]:
# Get specific vectors by their keys
vector_keys = ["ganga-river", "brahmaputra-river"]

try:
    response = s3vectors_client.get_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        keys=vector_keys,
        returnData=True,
        returnMetadata=True
    )
    
    print("✅ Vectors retrieved successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Display retrieved vectors
    if 'vectors' in response:
        print(f"\n📄 Retrieved Vectors ({len(response['vectors'])} found):")
        for vector in response['vectors']:
            key = vector.get('key', 'Unknown')
            metadata = vector.get('metadata', {})
            title = metadata.get('title', 'No title')
            region = metadata.get('region', 'No region')
            vector_dim = len(vector.get('data', {}).get('float32', []))
            print(f"  🌊 {key}: {title}")
            print(f"      📍 Region: {region} ({vector_dim}D vector)")
    
except Exception as e:
    print(f"❌ Error retrieving vectors: {e}")

✅ Vectors retrieved successfully!
{
  "ResponseMetadata": {
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 13 Aug 2025 05:15:05 GMT",
      "server": "uvicorn",
      "content-length": "32905",
      "content-type": "application/json"
    },
    "RetryAttempts": 0
  },
  "vectors": [
    {
      "key": "ganga-river",
      "data": {
        "float32": [
          0.04290274530649185,
          0.07912567257881165,
          -0.17291699349880219,
          -0.0007869930122978985,
          0.036546651273965836,
          0.030878186225891113,
          0.003549055429175496,
          0.04689719155430794,
          0.01741626113653183,
          0.001391763798892498,
          0.052074551582336426,
          0.015848932787775993,
          0.11315764486789703,
          0.0031625088304281235,
          0.039050422608852386,
          -0.05309721454977989,
          0.01617870107293129,
          -0.03039400465786457,
          -0.007509204093366861,
          0.07134

## 13. Test ListVectors

List all vectors in the index.

In [ ]:
# List all vectors in the index
try:
    response = s3vectors_client.list_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        maxResults=10
    )
    
    print("✅ Vectors listed successfully!")
    print(json.dumps(response, indent=2, default=str))
    
    # Display vector list
    if 'vectors' in response:  # Changed from 'vectorKeys' to 'vectors' for boto3 compatibility  
        print(f"\n📋 Vector Keys ({len(response['vectors'])} found):")
        for vector in response['vectors']:
            key = vector.get('key', 'Unknown')
            print(f"  🔑 {key}")
    
except Exception as e:
    print(f"❌ Error listing vectors: {e}")

## 14. Test DeleteVectorBucketPolicy

Remove the bucket policy we set earlier.

In [ ]:
# Delete bucket policy
try:
    response = s3vectors_client.delete_vector_bucket_policy(
        vectorBucketName=bucket_name
    )
    
    print("✅ Bucket policy deleted successfully!")
    print(json.dumps(response, indent=2, default=str))
    
except Exception as e:
    print(f"❌ Error deleting bucket policy: {e}")

## 15. Test DeleteVectors

Delete specific vectors from the index.

In [ ]:
# Delete specific vectors from the index
vectors_to_delete = ["krishna-river"]  # Delete one river for testing

try:
    response = s3vectors_client.delete_vectors(
        vectorBucketName=bucket_name,
        indexName=index_name,
        keys=vectors_to_delete
    )
    
    print("✅ Vectors deleted successfully!")
    print(json.dumps(response, indent=2, default=str))
    print(f"🗑️ Deleted {len(vectors_to_delete)} vectors: {', '.join(vectors_to_delete)}")
    
except Exception as e:
    print(f"❌ Error deleting vectors: {e}")

## 16. Clean Up - Delete Index

Delete the index we created for testing.

In [ ]:
# Delete the index
try:
    response = s3vectors_client.delete_index(
        vectorBucketName=bucket_name,
        indexName=index_name
    )
    
    print("✅ Index deleted successfully!")
    print(json.dumps(response, indent=2, default=str))
    
except Exception as e:
    print(f"❌ Error deleting index: {e}")

## 17. Clean Up - Delete Vector Bucket

Delete the vector bucket we created for testing.

In [ ]:
# Delete the vector bucket
try:
    response = s3vectors_client.delete_vector_bucket(
        vectorBucketName=bucket_name
    )
    
    print("✅ Vector bucket deleted successfully!")
    print(json.dumps(response, indent=2, default=str))
    
except Exception as e:
    print(f"❌ Error deleting bucket: {e}")

## 🎉 S3 Vectors Testing Complete!

This notebook demonstrates S3 Vectors API operations using the boto3 SDK with real text embeddings:

### ✅ Core Operations Tested
- **Bucket Operations**: CreateVectorBucket, ListVectorBuckets, DeleteVectorBucket
- **Policy Operations**: PutVectorBucketPolicy, GetVectorBucketPolicy, DeleteVectorBucketPolicy  
- **Index Operations**: CreateIndex, ListIndexes, DeleteIndex
- **Vector Operations**: PutVectors, GetVectors, ListVectors, QueryVectors, DeleteVectors

### 🧠 Key Features Demonstrated
- **Real Text Embeddings**: Using text-embedding-nomic-embed-text-v1.5 model (768 dimensions)
- **Semantic Search**: Query vectors with natural language questions
- **Metadata Filtering**: Filter results by category, region, and other attributes
- **Similarity Ranking**: Results ranked by cosine distance with relevance scoring

### 🌊 Sample Data
- Rivers of India knowledge base with geographic, religious, and economic information
- Semantic understanding of concepts like "sacred", "agriculture", "hydropower"
- Real-world use case demonstrating vector search capabilities

All operations work with native boto3 SDK experience and proper error handling! 🚀